# Importing Header and SoilPrep

In [1]:
import import_ipynb 
from Header import *

importing Jupyter notebook from Header.ipynb


In [2]:
import import_ipynb
from SoilPrep import * 

importing Jupyter notebook from SoilPrep.ipynb


## Loading Data and MetaData

In [3]:
import import_ipynb
from LoadDataMetaData import *

importing Jupyter notebook from LoadDataMetaData.ipynb


In [4]:
import os
os.system('say "your Meta Tree started building"')
import time

In [5]:
# prepare_spec = ['none', 'fod2', 'continuum', 'log']
# prepare_target = ['none', 'minmax']

def find_X(p, n):
    if p == 'fod2':
        X = fod_sampled[n]
    elif p == 'continuum':
        X = sampled_cr[n]
    elif p == 'log':
        X = sampled_log[n]
    else:
        X = sampled_spec[n]
    return X

def find_spec(p, n, m):
    if n == 0:
        if m == 'randomforest' or m == 'cubist' or m == 'gbrt':
            spec = find_X(p, 100)
        else:
            spec = find_X(p,n)            
    else:
        spec = find_X(p,n)
    return spec    


def find_y(t):
    i = target_names.index(t) 
    y = T[i]
    return y
        
        

## Recording Hypertuning Parameters

In [6]:
from sklearn.metrics import make_scorer, r2_score

model ={}
param_grid ={}
model['mult'] = LinearRegression()
param_grid['mult'] = { 'fit_intercept': [True, False] }

model['ridge'] = KernelRidge()
param_grid['ridge']={'alpha': [ 0.00001,0.00002, 0.00005, 0.0001, 0.0002, 0.0005, 0.001, 0.002, \
                               0.005,  0.01, 0.05, 0.1, 0.5, 1]}

model['plsr'] = PLSRegression()
param_grid['plsr']={'n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}


model['cubist'] = Cubist(random_state = 42)  
param_grid['cubist'] = { 'n_committees': [5, 10, 15, 20], 'n_rules': [10, 20, 30, 40, 50] }
                        

model['gbrt'] = GradientBoostingRegressor(random_state = 42)
param_grid['gbrt'] = {
    'n_estimators': [30,40,50],        # Number of boosting stages to be used
    'learning_rate': [0.01, 0.1],     # Step size shrinkage used in updating weights
    'max_depth': [3, 4, 5]                # Maximum depth of individual trees
#     'min_samples_split': [2, 3],        # Minimum number of samples required to split an internal node
#     'min_samples_leaf': [1, 2],         # Minimum number of samples required to be at a leaf node
}

model['svr'] = SVR()
param_grid['svr'] = {
    'C': [0.1, 1, 10],              # Regularization parameter
    'kernel': ['linear', 'rbf'],   # Kernel function
    'gamma': [0.01, 0.1, 1],      # Kernel coefficient (only for 'rbf' kernel)
}


### Finding Hypertuned Parameters

In [7]:
# def find_core_model_param_on (m, t, p, tp, n):
#     Model = model[m]
#     X = find_spec(p,n,m)
#     y = find_y(t, tp)
    
#     row, col = X.shape

#     scorer = make_scorer(mean_squared_error, greater_is_better=False)
#     cv = RepeatedKFold(n_splits=row, n_repeats=1, random_state=3)
        
#     grid_search = GridSearchCV(Model, param_grid=param_grid[m], cv=cv,\
#                                 scoring= scorer, refit=True, verbose=1, error_score='raise', n_jobs=-1)
#     grid_search.fit(X,y)
    
#     return (grid_search.best_params_)

def find_tuned_param_on (X_train, y_train, m):
    Model = model[m]
    X = X_train
    y = y_train
    
    row, col = X.shape

    scorer = make_scorer(mean_squared_error, greater_is_better=False)
    cv = RepeatedKFold(n_splits=row, n_repeats=1, random_state=3)
        
    grid_search = GridSearchCV(Model, param_grid=param_grid[m], cv=cv,\
                                scoring= scorer, refit=True, verbose=1, error_score='raise', n_jobs=-1)
    grid_search.fit(X,y)
    
    return (grid_search.best_params_)
    

### Creating Hypertuned Models

In [8]:
def create_core_model_on (m, BP):
    if m == 'mult':
        Model = LinearRegression()
        
    elif m == 'ridge':
        Model =  KernelRidge(alpha = BP['alpha'])
        
    elif m == 'plsr':
        Model = PLSRegression(n_components = BP['n_components'])
        
    elif m == 'cubist':
        Model =  Cubist( n_committees = BP['n_committees'], n_rules = BP['n_rules'], random_state = 42) 
    
    elif m == 'gbrt':
        Model = GradientBoostingRegressor(n_estimators = BP['n_estimators'], learning_rate = BP['learning_rate'], \
                                          max_depth = BP['max_depth'],  random_state = 42)
    elif m == 'svr':
        Model = SVR(C = BP['C'], kernel = BP['kernel'], gamma = BP['gamma'])
        
    return Model    
            

In [9]:
# def build_BPtree_for(method_name):
#     tree ={}
#     start = time.time()
#     m = method_name
#     #-- code to build tree----
#     for t in target_names:
#         print('BPtree for: '+ m +' ------> running on: ' + t)
#         tree[t] ={}
#         for tp in prepare_target:
#             tree[t][tp] ={}
#             for p in prepare_spec:
#                 tree[t][tp][p] ={}
#                 #---- find appropriate initial core model parameters-----------------
#                 if m == 'cubist' or m == 'gbrt':
#                     BP100 = find_core_model_param_on (m, t, p, tp, 100)
                
#                 for n in nbands_sampling: 
#                     tree[t][tp][p][n] ={}
#                     Y = tree[t][tp][p][n]
#                     if m == 'cubist' or m == 'gbrt':
#                         Y['BP'] = BP100
#                     else:    
#                         BP = find_core_model_param_on (m, t, p, tp, n)
#                         print('n_band:', n, '---> target:', t )
#                         Y['BP'] = BP
#     end = time.time() 
#     print('End time - Start time =', (end-start)) 
    
#     return (tree.copy())  

# def L1_y_pred (spectra, target, method, bp):
#     m = method
#     (r,c) = spectra.shape
    
#     Y_test = target.copy()
#     Y_pred = target.copy()
    
#     for i in range (0,r): 
#         #print('i: ', i)
#         full_spec = spectra.copy()
#         X_train = full_spec.drop(full_spec.index[i], axis=0)
#         X_test = full_spec.iloc[[i],:].copy()
#         full_tar = target.copy()
#         y_train = full_tar.drop(full_tar.index[i], axis=0)
#         y_test = full_tar.iloc[i].copy() 
        
#         #---- Model Creation, fitting, and predictions--------
#         Model = create_core_model_on (m, bp)
#         Model.fit(X_train, y_train)
#         y_pred = Model.predict(X_test)
#         Y_pred.iloc[i] = y_pred
        
#         return (Y_pred)      

## Building HMtree and BPtree

In [10]:
ml_methods = ['mult', 'ridge', 'plsr', 'svr', 'cubist', 'gbrt']

In [11]:
def L1_y_pred (spectra, target, method, bp):
    m = method
    (r,c) = spectra.shape
    
    Y_test = target.copy()
    Y_pred = target.copy()
    #print('r:', r)
    for i in range (0, r): 
        #print('i: ', i)
        full_spec = spectra.copy()
        X_train = full_spec.drop(full_spec.index[i], axis=0)
        X_test = full_spec.iloc[[i],:].copy()
        full_tar = target.copy()
        y_train = full_tar.drop(full_tar.index[i], axis=0)
        y_test = full_tar.iloc[i].copy()        
        #---- Model Creation, fitting, and predictions--------
        Model = create_core_model_on (m, bp)
        Model.fit(X_train, y_train)
        y_pred = Model.predict(X_test)
        Y_pred.iloc[i] = y_pred
        
    return Y_pred      

In [12]:
# m = 'mult'
# spec = find_spec('none', 10, m)
# y = find_y('Sand')

# bp = find_tuned_param_on (spec, y, m )

# L1y_pred = L1_y_pred (spec, y, m, bp)

In [13]:
def build_HMtree_on (method_name, target):
    
    tree ={}
    
    start = time.time()
    m = method_name
    t = target
    
    print('tree for: '+ m +' ------> running on: ' + t)
    
    for tp in prepare_target:
        tree[tp] ={}        
        for p in prepare_spec:
            tree[tp][p] ={}            
            for n in nbands_sampling:
                tree[tp][p][n] ={}
                Y = tree[tp][p][n]
                                           
                #------ setting spec to appropriate (sampled) spectra----                
                spec = find_spec(p, n, m)                
                                        
                #---- target selection and normalization ---
                y = find_y(t)
                                
                #---- performing train-test split----------------------
                X_train, X_test, y_train, y_test = train_test_split(spec, y, test_size= 0.3, random_state=42)
                            
                #----- hypertuning parameter, model creation, fitting and prediction ----                
                
                #bp = find_tuned_param_on (X_train, y_train, m)
                bp = find_tuned_param_on (spec, y, m)
                
                Y['bp'] = bp
                print('Method: '+ m + '---> Target: '+ t+ '----> SpecProc: ' +p+ ' ---> n_band: ', n)
                Model = create_core_model_on (m, bp)
                Model.fit(X_train, y_train)
                
                y_pred = Model.predict(X_test)
                yhat_pred = Model.predict(X_train)
                    
                #----- Data for Model accuracy and plotting -----------
                Y['X_test'] = X_test                
                Y['y_test'] = y_test
                Y['y_testP'] = y_pred
                
                Y['X_train'] = X_train
                Y['y_train'] = y_train
                Y['y_trainP'] = yhat_pred
                    
                Y['iqrp_test'] = find_iqrp(y_pred, y_test)
                Y['r2_test'] = find_r2(y_pred, y_test)
                Y['rpd_test'] = find_rpd(y_pred, y_test)                 
                
                Y['r2_train'] = find_r2(yhat_pred, y_train)
                
                #------- L1 out prediction on test data -----------------
                L1y_pred = L1_y_pred (spec, y, m, bp)
                Y['L1y_testP'] = L1y_pred
                Y['L1y_test'] = y
                 
                
                Y['L1iqrp_test'] = find_iqrp(L1y_pred, y)
                Y['L1r2_test'] = find_r2(L1y_pred, y)
                Y['L1rpd_test'] = find_rpd(L1y_pred, y) 
                
                                                                         
    end = time.time()                            
    os.system('say "your hyper tuned meta tree finished for one more method"')                            

    print('End time - Start time =', (end-start)) 
    
    return tree.copy()

In [14]:
def build_HMtree_for (method_name):
    tree ={}
    
    start = time.time()
    m = method_name
    for t in target_names:
        #print('tree for: '+ m +' ------> running on: ' + t)
        tree[t] = build_HMtree_on (m, t)
        
    end = time.time()                            
    os.system('say "your meta tree finished for one more method"')                            
    print('End time - Start time =', (end-start))     
    return tree.copy()

In [15]:
def best_score_on (HMtree, target, method, scorer):
    t=target
    m= method
    
    best_score = -1
    #best_n_comp = 'NA'
    
    
    for tp in prepare_target:
        for p in prepare_spec:
            for n in nbands_sampling:
                Y = HMtree[m][t][tp][p][n]
                #r2_train = Y['r2_train']
                #r2_test = Y['r2_test']
                #L1r2_test = Y['L1r2_test']
                 
                    
                if scorer == 'iqrp':
                    cur_score = Y['iqrp_test']
                elif scorer == 'L1iqrp':
                    cur_score = Y['L1iqrp_test']
                elif scorer == 'r2':
                    cur_score = Y['r2_test'] 
                else:
                    cur_score = Y['L1r2_test']  
                    
                if cur_score > best_score:
                    best_score = cur_score
                    best_tp = tp
                    best_n = n
                    best_p = p
                            
    param_list = [scorer, np.round(best_score,2), 'Spec:', best_p, 'bands:', best_n]                                 
    return (param_list)          

In [16]:
def best_score_for (HMtree, target, scorer):
    
    for method in ml_methods:
        param_list= best_score_on (HMtree, target, method, scorer)
        print('For:'+target+'->', param_list, ':'+method)
     
    return

In [17]:
with open ('HMtree.pickle', 'rb') as file:
    HMtree = pickle.load(file)
 

## Creating different branches of Mtree (for separate methods)

In [18]:
#HMtree = {}


In [19]:
#HMtree['mult'] = build_HMtree_for('mult')

In [20]:
# with open ('HMtree.pickle', 'wb') as file:
#     pickle.dump(HMtree, file)

In [21]:
# HMtree['plsr'] = build_HMtree_for('plsr')
# HMtree['svr'] = build_HMtree_for('svr')
# HMtree['ridge'] = build_HMtree_for('ridge')

In [22]:
# with open ('HMtree.pickle', 'wb') as file:
#     pickle.dump(HMtree, file)
        

In [23]:
# HMtree['cubist'] = build_HMtree_for('cubist')

In [24]:
# with open ('HMtree.pickle', 'wb') as file:
#     pickle.dump(HMtree, file)
     

In [25]:
#HMtree['gbrt'] = build_HMtree_for('gbrt')

In [26]:
# with open ('HMtree.pickle', 'wb') as file:
#     pickle.dump(HMtree, file)
     

In [27]:
#Plotting Model Accuracy (ipywidgets)
def plot_model_acc (method, target, spec_preprocessing, n_bands):
    
    m = method
    t = target
    
    i = target_names.index(target)    
    
    p = spec_preprocessing
    n = n_bands
    tp = 'none'
    
    Y = HMtree[m][t][tp][p][n]
    
    y_test = Y['y_test']
    y_pred = Y['y_testP']
    L1y_pred = Y['L1y_testP']
    L1y_test = Y['L1y_test']
    

    
    if m == 'plsr':
        y_pred = y_pred[:,0]
        #L1y_pred = L1y_pred[:,0]

    
    iqrp_test = Y['iqrp_test']
    r2_test = Y['r2_test']
    rpd_test = Y['rpd_test']
    
    L1iqrp_test = Y['L1iqrp_test']
    L1r2_test = Y['L1r2_test']
    L1rpd_test = Y['L1rpd_test']
    
    y_tp = pd.DataFrame({'actual':y_test.values, 'predic': y_pred})
    z = np.polyfit(y_test, y_pred, 1)
    
    L1y_tp = pd.DataFrame({'actual':L1y_test.values, 'predic': L1y_pred})
    L1z = np.polyfit(L1y_test, L1y_pred, 1)
    
    fig, axes = plt.subplots(2,2, figsize=(18,16))
    
    #with plt.style.context(('ggplot')): ---- PLOT of test-prediction --------------------------------------
    y_tp.plot.scatter(ax= axes[0][0], x="actual", y="predic", alpha=0.8, color = clr[i], edgecolors='k')
    axes[0][0].plot(y_test, np.polyval(z, y_test),  c='blue', linewidth=1)
    axes[0][0].plot(y_test, y_test, color='green', linewidth=1)
    axes[0][0].tick_params(axis='both', labelsize=10)
    axes[0][0].text(0.05, 0.95, target_names[i]+' (Train-Test Score)', transform=axes[0][0].transAxes, fontsize = 20, color = clr[i])
    axes[0][0].text(0.05, 0.90, 'IQRP ={:.2f}'.format(iqrp_test), transform=axes[0][0].transAxes, fontsize = 16)
    axes[0][0].text(0.05, 0.85, 'RPD ={:.2f}'.format(rpd_test), transform=axes[0][0].transAxes, fontsize = 16)
    axes[0][0].text(0.05, 0.80, 'R2 ={:.2f}'.format(np.round(r2_test,3)), transform=axes[0][0].transAxes, fontsize = 16)
    axes[0][0].text(0.95, 0.15, 'Method: '+method, transform=axes[0][0].transAxes, 
                    horizontalalignment='right', fontsize = 20)
    
    L1y_tp.plot.scatter(ax= axes[0][1], x="actual", y="predic", alpha=0.8, color = clr[i], edgecolors='k')
    axes[0][1].plot(L1y_test, np.polyval(L1z, L1y_test),  c='blue', linewidth=1)
    axes[0][1].plot(L1y_test, L1y_test, color='green', linewidth=1)
    axes[0][1].tick_params(axis='both', labelsize=10)
    axes[0][1].text(0.05, 0.95, target_names[i]+' (L1-out Score)', transform=axes[0][1].transAxes, fontsize = 20, color = clr[i])
    axes[0][1].text(0.05, 0.90, 'IQRP ={:.2f}'.format(L1iqrp_test), transform=axes[0][1].transAxes, fontsize = 16)
    axes[0][1].text(0.05, 0.85, 'RPD ={:.2f}'.format(L1rpd_test), transform=axes[0][1].transAxes, fontsize = 16)
    axes[0][1].text(0.05, 0.80, 'R2 ={:.2f}'.format(np.round(L1r2_test,3)), transform=axes[0][1].transAxes, fontsize = 16)
    axes[0][1].text(0.95, 0.15, 'Method: '+method, transform=axes[0][1].transAxes, 
                    horizontalalignment='right', fontsize = 20)

    
     #------------------------ Ploting R2 Vs N_bands-----------------------------------
    
    #--- data for creating n_bands vs r2 scores plot -------------------------------
    
    pY = HMtree[m][t]['none'][p]        
    X = nbands_sampling
    
    Yr2 = []
    L1Yr2 = []
    
    for j in X:
        Yr2.append(pY[j]['r2_test'])
        L1Yr2.append(pY[j]['L1r2_test'])  
   
    j = 0
    while j < len(Yr2): 
        if Yr2[j] <= 0:
            Yr2[j] = 0
        if L1Yr2[j] <= 0:
            L1Yr2[j] = 0
        j = j + 1
 
    #------------------------------ STEM PLOT ---- for accuracy Vs n_bands ---------------------------------- 
    
    axes[1][0].stem(X,Yr2)
    axes[1][0].tick_params(axis='both', labelsize=10)
    axes[1][0].text(0.00, 1.01,  target_names[i], transform=axes[1][0].transAxes,fontsize = 20, color = clr[i])
    axes[1][0].text(0.99, 1.01, 'Spec_prep: '+ p, transform=axes[1][0].transAxes, horizontalalignment='right', fontsize = 16)
    axes[1][0].text(0.17, 1.01, '(train-test)', transform=axes[1][0].transAxes, horizontalalignment='left', fontsize = 16)
    axes[1][0].text(0.60, -0.1, 'n_bands', transform=axes[1][0].transAxes, horizontalalignment='right', fontsize = 16)
    axes[1][0].text(-0.1, 0.5, 'R2 Scores', horizontalalignment='left', verticalalignment='center', \
                rotation='vertical', transform=axes[1][0].transAxes, fontsize = 16)
    
    
    axes[1][1].stem(X,L1Yr2)
    axes[1][1].tick_params(axis='both', labelsize=10)
    axes[1][1].text(0.00, 1.01,  target_names[i], transform=axes[1][1].transAxes,fontsize = 20, color = clr[i])
    axes[1][1].text(0.99, 1.01, 'Spec_prep: '+ p, transform=axes[1][1].transAxes, horizontalalignment='right', fontsize = 16)
    axes[1][1].text(0.17, 1.01, '(L1-out)', transform=axes[1][1].transAxes, horizontalalignment='left', fontsize = 16)
    axes[1][1].text(0.60, -0.1, 'n_bands', transform=axes[1][1].transAxes, horizontalalignment='right', fontsize = 16)
    axes[1][1].text(-0.1, 0.5, 'R2 Scores', horizontalalignment='left', verticalalignment='center', \
                rotation='vertical', transform=axes[1][1].transAxes, fontsize = 16)
        
   
    return    

In [28]:
#ml_methods = ['mult', 'ridge', 'plsr', 'svr', 'cubist', 'gbrt']

ml_methods = ['mult', 'plsr', 'svr', 'ridge', 'cubist','gbrt']

In [29]:
best_score_for (HMtree, 'Sand', 'r2')

For:Sand-> ['r2', 0.64, 'Spec:', 'log', 'bands:', 7] :mult
For:Sand-> ['r2', 0.67, 'Spec:', 'none', 'bands:', 7] :plsr
For:Sand-> ['r2', 0.62, 'Spec:', 'log', 'bands:', 0] :svr
For:Sand-> ['r2', 0.6, 'Spec:', 'log', 'bands:', 7] :ridge
For:Sand-> ['r2', 0.62, 'Spec:', 'none', 'bands:', 7] :cubist
For:Sand-> ['r2', 0.53, 'Spec:', 'fod2', 'bands:', 17] :gbrt


In [30]:
best_score_for (HMtree, 'Sand', 'L1r2')

For:Sand-> ['L1r2', 0.54, 'Spec:', 'log', 'bands:', 7] :mult
For:Sand-> ['L1r2', 0.57, 'Spec:', 'log', 'bands:', 7] :plsr
For:Sand-> ['L1r2', 0.53, 'Spec:', 'log', 'bands:', 0] :svr
For:Sand-> ['L1r2', 0.55, 'Spec:', 'log', 'bands:', 7] :ridge
For:Sand-> ['L1r2', 0.59, 'Spec:', 'log', 'bands:', 7] :cubist
For:Sand-> ['L1r2', 0.59, 'Spec:', 'fod2', 'bands:', 39] :gbrt


In [31]:
best_score_for (HMtree, 'Silt', 'r2')

For:Silt-> ['r2', 0.36, 'Spec:', 'none', 'bands:', 5] :mult
For:Silt-> ['r2', 0.55, 'Spec:', 'none', 'bands:', 11] :plsr
For:Silt-> ['r2', 0.35, 'Spec:', 'log', 'bands:', 70] :svr
For:Silt-> ['r2', 0.41, 'Spec:', 'none', 'bands:', 7] :ridge
For:Silt-> ['r2', 0.41, 'Spec:', 'fod2', 'bands:', 5] :cubist
For:Silt-> ['r2', 0.54, 'Spec:', 'fod2', 'bands:', 19] :gbrt


In [32]:
best_score_for (HMtree, 'Silt', 'L1r2')

For:Silt-> ['L1r2', 0.3, 'Spec:', 'none', 'bands:', 5] :mult
For:Silt-> ['L1r2', 0.35, 'Spec:', 'fod2', 'bands:', 29] :plsr
For:Silt-> ['L1r2', 0.31, 'Spec:', 'log', 'bands:', 90] :svr
For:Silt-> ['L1r2', 0.32, 'Spec:', 'none', 'bands:', 5] :ridge
For:Silt-> ['L1r2', 0.42, 'Spec:', 'fod2', 'bands:', 37] :cubist
For:Silt-> ['L1r2', 0.42, 'Spec:', 'fod2', 'bands:', 70] :gbrt


In [33]:
best_score_for (HMtree, 'Clay', 'r2')

For:Clay-> ['r2', 0.75, 'Spec:', 'log', 'bands:', 11] :mult
For:Clay-> ['r2', 0.72, 'Spec:', 'log', 'bands:', 39] :plsr
For:Clay-> ['r2', 0.68, 'Spec:', 'log', 'bands:', 70] :svr
For:Clay-> ['r2', 0.72, 'Spec:', 'log', 'bands:', 9] :ridge
For:Clay-> ['r2', 0.72, 'Spec:', 'log', 'bands:', 7] :cubist
For:Clay-> ['r2', 0.64, 'Spec:', 'fod2', 'bands:', 7] :gbrt


In [34]:
best_score_for (HMtree, 'Clay', 'L1r2')

For:Clay-> ['L1r2', 0.61, 'Spec:', 'log', 'bands:', 7] :mult
For:Clay-> ['L1r2', 0.66, 'Spec:', 'log', 'bands:', 15] :plsr
For:Clay-> ['L1r2', 0.63, 'Spec:', 'log', 'bands:', 0] :svr
For:Clay-> ['L1r2', 0.6, 'Spec:', 'log', 'bands:', 90] :ridge
For:Clay-> ['L1r2', 0.67, 'Spec:', 'log', 'bands:', 21] :cubist
For:Clay-> ['L1r2', 0.63, 'Spec:', 'fod2', 'bands:', 90] :gbrt


In [35]:
best_score_for (HMtree, 'TOC', 'r2')

For:TOC-> ['r2', 0.47, 'Spec:', 'log', 'bands:', 9] :mult
For:TOC-> ['r2', 0.6, 'Spec:', 'log', 'bands:', 39] :plsr
For:TOC-> ['r2', 0.58, 'Spec:', 'log', 'bands:', 0] :svr
For:TOC-> ['r2', 0.6, 'Spec:', 'log', 'bands:', 0] :ridge
For:TOC-> ['r2', 0.62, 'Spec:', 'fod2', 'bands:', 45] :cubist
For:TOC-> ['r2', 0.64, 'Spec:', 'none', 'bands:', 0] :gbrt


In [36]:
best_score_for (HMtree, 'TOC', 'L1r2')

For:TOC-> ['L1r2', 0.42, 'Spec:', 'log', 'bands:', 19] :mult
For:TOC-> ['L1r2', 0.44, 'Spec:', 'log', 'bands:', 25] :plsr
For:TOC-> ['L1r2', 0.45, 'Spec:', 'log', 'bands:', 0] :svr
For:TOC-> ['L1r2', 0.45, 'Spec:', 'log', 'bands:', 0] :ridge
For:TOC-> ['L1r2', 0.5, 'Spec:', 'log', 'bands:', 39] :cubist
For:TOC-> ['L1r2', 0.54, 'Spec:', 'fod2', 'bands:', 50] :gbrt


In [37]:
best_score_for (HMtree, 'CaCO3', 'r2')

For:CaCO3-> ['r2', 0.67, 'Spec:', 'log', 'bands:', 7] :mult
For:CaCO3-> ['r2', 0.67, 'Spec:', 'log', 'bands:', 7] :plsr
For:CaCO3-> ['r2', 0.63, 'Spec:', 'none', 'bands:', 0] :svr
For:CaCO3-> ['r2', 0.65, 'Spec:', 'none', 'bands:', 9] :ridge
For:CaCO3-> ['r2', 0.69, 'Spec:', 'fod2', 'bands:', 60] :cubist
For:CaCO3-> ['r2', 0.59, 'Spec:', 'fod2', 'bands:', 35] :gbrt


In [38]:
best_score_for (HMtree, 'CaCO3', 'L1r2')

For:CaCO3-> ['L1r2', 0.56, 'Spec:', 'log', 'bands:', 7] :mult
For:CaCO3-> ['L1r2', 0.61, 'Spec:', 'none', 'bands:', 10] :plsr
For:CaCO3-> ['L1r2', 0.58, 'Spec:', 'log', 'bands:', 100] :svr
For:CaCO3-> ['L1r2', 0.6, 'Spec:', 'log', 'bands:', 0] :ridge
For:CaCO3-> ['L1r2', 0.66, 'Spec:', 'fod2', 'bands:', 27] :cubist
For:CaCO3-> ['L1r2', 0.66, 'Spec:', 'fod2', 'bands:', 27] :gbrt


In [39]:
ipywidgets.interact( plot_model_acc, target = target_names, method = ml_methods, \
                    spec_preprocessing = prepare_spec,  \
                    target_preprocessing = prepare_target, n_bands = nbands_sampling )

interactive(children=(Dropdown(description='method', options=('mult', 'plsr', 'svr', 'ridge', 'cubist', 'gbrt'…

<function __main__.plot_model_acc(method, target, spec_preprocessing, n_bands)>

In [40]:
def create_best_model_for (HMtree,target_names):
    for target in target_names:
        
    
    

IndentationError: expected an indented block (12846923.py, line 5)

In [ ]:
#  def stable_HMtree_on (method_name, target, BPtree):
#     tree ={}
    
#     start = time.time()
#     m = method_name
#     t = target
    
#     print('stable tree for: '+ m +' ------> building on: ' + t)
    
#     for tp in prepare_target:
#         tree[tp] ={}
        
#         for p in prepare_spec:
#             tree[tp][p] ={}
            
#             for n in nbands_sampling:
#                 tree[tp][p][n] ={}
#                 Y = tree[tp][p][n]
                
                                           
#                 #------ setting spec to appropriate (sampled) spectra----
                
#                 spec = find_spec(p, n, m)                
                                        
#                 #---- target selection and normalization ---
#                 y = find_y(t)
                
                            
#                 #----- hypertuned model creation, L1 out fitting and predictions ----
                                
#                 bp = BPtree[m][t][tp][p][n]['BP']
#                 y_pred = L1_y_pred (spec, y, m, bp)
                    
#                 #----- Model accuracy evaluation and plotting datas-----------
                            
#                 Y['test'] = y_test
#                 Y['testP'] = y_pred                
                    
#                 Y['iqrp_test'] = find_iqrp(y_pred, y_test)
#                 Y['r2_test'] = find_r2(y_pred, y_test)
#                 Y['rpd_test'] = find_rpd(y_pred, y_test)                    
#                 Y['rmse_test'] = find_rmse(y_pred, y_test)
                                                                
                                                                         
#     end = time.time()                            
#     os.system('say "your stable hypertuned meta tree finished for one more method"')                            

#     print('End time - Start time =', (end-start)) 
    
#     return tree.copy()

# def stable_HMtree_for (method_name, BPtree):
#     tree ={}
    
#     start = time.time()
#     m = method_name
#     for t in target_names:
#         tree[t] = stable_HMtree_on (m, t, BPtree)
        
#     end = time.time()                            
#     os.system('say "your stable hypertuned meta tree finished for one more method"')                            
#     print('End time - Start time =', (end-start))     
#     return tree.copy()

In [ ]:
# with open ('HMtree.pickle', 'wb') as file:
#     pickle.dump(HMtree, file)
    
# with open ('BPtree.pickle', 'wb') as file:
#     pickle.dump(BPtree, file)  

In [ ]:
halt here

In [ ]:
#prepare_spec = ['none', 'fod2', 'cr', 'log']

In [ ]:
best_score_for (HMtree, 'Sand', 'r2')

In [ ]:
best_score_for (HMtree, 'Silt', 'r2')

In [ ]:
best_score_for (HMtree, 'Clay', 'r2')

In [ ]:
best_score_for (HMtree, 'TOC', 'r2')

In [ ]:
best_score_for (HMtree, 'CaCO3', 'r2')

## Plotting Model Accuracy (ipywidgets)

In [ ]:
ipywidgets.interact( plot_model_acc, target = target_names, method = ml_methods, \
                    spec_preprocessing = prepare_spec,  \
                    target_preprocessing = prepare_target, n_bands = nbands_sampling )

In [ ]:
# with open ('HMtree.pickle', 'wb') as file:
#     pickle.dump(HMtree, file)